<a href="https://colab.research.google.com/github/linDing-groups/Deep-4mCW2V/blob/W2V/Toxin.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import KFold
from keras.layers import Input, Dense, Conv1D, Flatten, MaxPooling1D, Conv2D, MaxPooling2D, AveragePooling2D, Dropout, Reshape, normalization
from keras.models import Model
import keras.backend as K
from keras.layers.recurrent import LSTM
import h5py
from sklearn import metrics
import random
from keras.regularizers import l1, l2
from tensorflow.python.keras.optimizers import TFOptimizer

In [ ]:
def precision(y_true, y_pred):
    # Calculates the precision
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    return precision

def recall(y_true, y_pred):
    # Calculates the recall
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    recall = true_positives / (possible_positives + K.epsilon())
    return recall

def f1(test_Y, pre_test_y):
    #calculate the F1-score
    Precision = precision(test_Y, pre_test_y)
    Recall = recall(test_Y, pre_test_y)
    f1 = 2 * ((Precision * Recall) / (Precision + Recall + K.epsilon()))
    return f1 

def TP(test_Y,pre_test_y):
    #calculate numbers of true positive samples
    TP = K.sum(K.round(K.clip(test_Y * pre_test_y, 0, 1)))#TP
    return TP

def FN(test_Y,pre_test_y):
     #calculate numbers of false negative samples
    TP = K.sum(K.round(K.clip(test_Y * pre_test_y, 0, 1)))#TP
    P=K.sum(K.round(K.clip(test_Y, 0, 1)))
    FN = P-TP #FN=P-TP
    return FN

def TN(test_Y,pre_test_y):
    #calculate numbers of True negative samples
    TN=K.sum(K.round(K.clip((test_Y-K.ones_like(test_Y))*(pre_test_y-K.ones_like(pre_test_y)), 0, 1)))#TN
    return TN

def FP(test_Y,pre_test_y):
    #calculate numbers of False positive samples
    N = (-1)*K.sum(K.round(K.clip(test_Y-K.ones_like(test_Y), -1, 0)))#N
    TN=K.sum(K.round(K.clip((test_Y-K.ones_like(test_Y))*(pre_test_y-K.ones_like(pre_test_y)), 0, 1)))#TN
    FP=N-TN
    return FP

def dnn_model(train_X, train_Y, test_X, test_Y, lr, epoch, batch_size):
    train_X = np.expand_dims(train_X, 2)
    test_X = np.expand_dims(test_X, 2)
    inputs = Input(shape = (train_X.shape[1], train_X.shape[2]))
    x = Conv1D(32, kernel_size = 2, strides = 1, padding = 'valid', activation = 'relu')(inputs)
    x = MaxPooling1D(pool_size = 2, strides = 2, padding = 'same')(x)
    x = Flatten()(x)
    x = Dropout(0.5)(x)
    x = Dense(32, activation = 'relu',kernel_regularizer = l2(1e-7))(x)
    x = Dense(16, activation = 'relu',kernel_regularizer = l2(1e-7))(x)
    x = Dense(8, activation = 'relu',kernel_regularizer = l2(1e-7))(x)
    predictions = Dense(1, activation = 'sigmoid')(x) 
    model = Model(inputs = inputs, outputs = predictions)
    print("model")
    model.compile(optimizer = 'RMSProp',
                  loss = 'mean_squared_error',
                  metrics = ['acc',precision,recall,f1,TP,FN,TN,FP])
    print("compile")
    result = model.fit(train_X, train_Y, epochs = epoch, batch_size = 32, validation_data = (test_X, test_Y), shuffle = True)
    model.save('/content/drive/MyDrive/hasan/Toxin_model') #save model
    pre_test_y = model.predict(test_X, batch_size = 50)
    pre_train_y = model.predict(train_X, batch_size = 50)
    test_auc = metrics.roc_auc_score(test_Y, pre_test_y)
    train_auc = metrics.roc_auc_score(train_Y, pre_train_y)
    print("train_auc: ", train_auc)
    print("test_auc: ", test_auc) 
    return test_auc, result
    

In [ ]:
# split data and output result
data = np.array(pd.read_csv("/content/drive/MyDrive/Fusion_snake.csv"))#inputfile
X1 = data[0:216, 1:]#216 is the number of positive samples in training set, '1' is the label of positive sample
Y1 = data[0:216, 0]#'0' is the label of negative sample
X2 = data[216:, 1:]
Y2 = data[216:, 0]
X = np.concatenate([X1, X2], 0)
Y = np.concatenate([Y1, Y2], 0)
#Y = Y.reshape((Y.shape[0], -1))
print (X)
print ("X.shape: ", X.shape)
print ("Y.shape: ", Y.shape)

lr = 0.05 #learning rate
epoch = 50
batch_size = 32
kf = KFold(n_splits = 5, shuffle = True, random_state = 20)
#kf = KFold(n_splits = 5, shuffle = False)
kf = kf.split(X)

test_aucs = []
for i, (train_fold, validate_fold) in enumerate(kf):
    print("\n\ni: ", i)
    test_auc = dnn_model(X[train_fold], Y[train_fold], X[validate_fold], Y[validate_fold], lr, epoch, batch_size)
    test_auc = test_aucs.append(test_auc)


[[-2.05138510e-02 -3.55731500e-01 -6.52737700e-02 ...  1.80000000e+01
   1.30611111e+02  2.13376340e+01]
 [ 1.86386440e-01 -2.27252830e-01 -3.44591350e-01 ...  6.00000000e+00
   8.26666670e+01  1.84613650e+01]
 [-1.33195920e-01 -1.70529300e-01 -5.33389820e-02 ...  2.00000000e+01
   1.74250000e+02  3.71103560e+01]
 ...
 [-6.61259500e-01  2.26810220e-02 -2.57873720e-02 ...  3.00000000e+00
   1.03333333e+02  1.62480970e+01]
 [-1.91675800e-01 -1.97554580e-01 -4.30163850e-02 ...  2.60000000e+01
   2.40153846e+02  3.39526630e+01]
 [-6.81259500e-01  2.36810220e-02 -2.59873720e-02 ...  4.00000000e+00
   3.60000000e+01  2.22549000e+00]]
X.shape:  (432, 660)
Y.shape:  (432,)


i:  0
model
compile
Epoch 1/50
11/11 [==============================] - 2s 59ms/step - loss: 0.3124 - acc: 0.5188 - precision: 0.5369 - recall: 0.7382 - f1: 0.5453 - TP: 11.9091 - FN: 3.9091 - TN: 4.3636 - FP: 11.1818 - val_loss: 0.2210 - val_acc: 0.5632 - val_precision: 0.4568 - val_recall: 0.6667 - val_f1: 0.5135 - val_T